# Single dish (GBT) conversion guide

In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

XRADIO version 0.0.45 already installed.


## Download dataset

https://open-bitbucket.nrao.edu/projects/CASA/repos/casatestdata/browse/unittest/tclean/venus_ephem_test.ms

In [2]:
import toolviper

toolviper.utils.data.download(file="sdimaging.ms")

[2024-12-06 17:17:59,373]  WARNING    viperlog:  File exists: /home/fedemp/ws_xradio_dev/venv_xradio_python39/lib/python3.9/site-packages/toolviper/utils/data/.dropbox 
[2024-12-06 17:17:59,374]     INFO    viperlog:  Updating file metadata information ...  
 

 Download List  
 ─────────────── 
  sdimaging.ms

[2024-12-06 17:18:00,886]     INFO    viperlog:  File exists: sdimaging.ms 


## Convert to Processing Set

In [3]:
from xradio.measurement_set import convert_msv2_to_processing_set

ms_file = "sdimaging.ms"

main_chunksize = {"time": 63}  # baseline, polarization
outfile = "sdimaging.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

[2024-12-06 17:18:03,074]     INFO    viperlog:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBS_MODE', 'OBSERVATION_ID', 'FIELD_ID'] 
[2024-12-06 17:18:03,081]     INFO    viperlog:  Number of partitions: 1 
[2024-12-06 17:18:03,083]     INFO    viperlog:  OBSERVATION_ID [0], DDI [0], STATE [0], FIELD [0], SCAN [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61] 


## Processing Set

In [4]:
from xradio.measurement_set import open_processing_set

outfile = "sdimaging.vis.zarr"
ps = open_processing_set(ps_store=outfile)
ps.summary()

,name,intents,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
0,sdimaging_0,"[OBSERVE_TARGET_ON_SOURCE, FREQUENCY_SWITCH_SIG]","(3843, 1, 1024, 2)","[XX, YY]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",spw_0,[FLS3a_0],[FLS3a_0],[],"[fk5, 0h00m00.00s, 0d00m00.00s]",1.419395e+09,1.421893e+09


In [5]:
ps.keys()

dict_keys(['sdimaging_0'])

In [6]:
msv4 = ps["sdimaging_0"]
msv4

<xarray.MeasurementSetXds> Size: 71MB
Dimensions:                     (antenna_name: 1, time: 3843, frequency: 1024,
                                 polarization: 2)
Coordinates:
  * antenna_name                (antenna_name) <U3 12B 'GBT'
    scan_number                 (time) int64 31kB dask.array<chunksize=(3843,), meta=np.ndarray>
    telescope_name              (antenna_name) <U3 12B dask.array<chunksize=(1,), meta=np.ndarray>
  * time                        (time) float64 31kB 1.069e+09 ... 1.069e+09
  * frequency                   (frequency) float64 8kB 1.419e+09 ... 1.422e+09
  * polarization                (polarization) <U2 16B 'XX' 'YY'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, antenna_name) float64 31kB dask.array<chunksize=(63, 1), meta=np.ndarray>
    FLAG                        (time, antenna_name, frequency, polarization) bool 8MB dask.array<chunksize=(63, 1, 1024, 2), meta=np.ndarray>
    SPECTRUM                    (time, antenna_name, frequency, polarization) float32 31MB dask.array<chunksize=(63, 1, 1024, 2), meta=np.ndarray>
    TIME_CENTROID               (time, antenna_name) float64 31kB dask.array<chunksize=(63, 1), meta=np.ndarray>
    WEIGHT                      (time, antenna_name, frequency, polarization) float32 31MB dask.array<chunksize=(63, 1, 1024, 2), meta=np.ndarray>
Attributes:
    creation_date:           2024-12-06T16:18:03.098049+00:00
    data_groups:             {'base': {'correlated_data': 'SPECTRUM', 'flag':...
    observation_info:        {'observer': ['AGBT02A_031_12'], 'project': '', ...
    partition_info:          {'field_name': ['FLS3a_0'], 'intents': ['OBSERVE...
    processor_info:          {'sub_type': '', 'type': ''}
    schema_version:          4.0.-9992
    type:                    spectrum
    xradio_version:          0.0.45
    antenna_xds:             <xarray.Dataset> Size: 192B\nDimensions:        ...
    system_calibration_xds:  <xarray.Dataset> Size: 154kB\nDimensions:       ...
    weather_xds:             <xarray.Dataset> Size: 96B\nDimensions:         ...
    pointing_xds:            <xarray.Dataset> Size: 92kB\nDimensions:        ...

In [7]:
ps["sdimaging_0"].attrs["antenna_xds"]

<xarray.Dataset> Size: 192B
Dimensions:                 (antenna_name: 1, cartesian_pos_label: 3,
                             receptor_label: 2)
Coordinates:
  * antenna_name            (antenna_name) <U3 12B 'GBT'
  * cartesian_pos_label     (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
    mount                   (antenna_name) <U6 24B dask.array<chunksize=(1,), meta=np.ndarray>
    polarization_type       (antenna_name, receptor_label) <U1 8B dask.array<chunksize=(1, 2), meta=np.ndarray>
  * receptor_label          (receptor_label) <U5 40B 'pol_0' 'pol_1'
    station                 (antenna_name) <U9 36B dask.array<chunksize=(1,), meta=np.ndarray>
    telescope_name          (antenna_name) <U3 12B dask.array<chunksize=(1,), meta=np.ndarray>
Data variables:
    ANTENNA_DISH_DIAMETER   (antenna_name) float64 8B dask.array<chunksize=(1,), meta=np.ndarray>
    ANTENNA_POSITION        (antenna_name, cartesian_pos_label) float64 24B dask.array<chunksize=(1, 3), meta=np.ndarray>
    ANTENNA_RECEPTOR_ANGLE  (antenna_name, receptor_label) float64 16B dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes:
    overall_telescope_name:  GBT
    relocatable_antennas:    False
    type:                    antenna

In [8]:
msv4.attrs["weather_xds"]

<xarray.Dataset> Size: 96B
Dimensions:         (station_name: 1, time_weather: 1, antenna_name: 1)
Coordinates:
  * antenna_name    (antenna_name) <U3 12B 'GBT'
  * station_name    (station_name) <U9 36B 'GREENBANK'
  * time_weather    (time_weather) float64 8B 4.576e+09
Data variables:
    PRESSURE        (station_name, time_weather) float64 8B dask.array<chunksize=(1, 1), meta=np.ndarray>
    REL_HUMIDITY    (station_name, time_weather) float64 8B dask.array<chunksize=(1, 1), meta=np.ndarray>
    TEMPERATURE     (station_name, time_weather) float64 8B dask.array<chunksize=(1, 1), meta=np.ndarray>
    WIND_DIRECTION  (station_name, time_weather) float64 8B dask.array<chunksize=(1, 1), meta=np.ndarray>
    WIND_SPEED      (station_name, time_weather) float64 8B dask.array<chunksize=(1, 1), meta=np.ndarray>
Attributes:
    type:     weather

In [9]:
msv4.attrs["system_calibration_xds"]

<xarray.Dataset> Size: 154kB
Dimensions:         (antenna_name: 1, time_cal: 3843, receptor_label: 2,
                     frequency_cal: 1)
Coordinates:
  * antenna_name    (antenna_name) <U3 12B 'GBT'
  * frequency_cal   (frequency_cal) int64 8B 0
  * receptor_label  (receptor_label) int64 16B 0 1
  * time_cal        (time_cal) float64 31kB 4.575e+09 4.575e+09 ... 4.575e+09
Data variables:
    TCAL            (antenna_name, time_cal, receptor_label, frequency_cal) float64 61kB dask.array<chunksize=(1, 3843, 2, 1), meta=np.ndarray>
    TSYS            (antenna_name, time_cal, receptor_label, frequency_cal) float64 61kB dask.array<chunksize=(1, 3843, 2, 1), meta=np.ndarray>
Attributes:
    type:     system_calibration

In [10]:
msv4.attrs["pointing_xds"]

<xarray.Dataset> Size: 92kB
Dimensions:              (time_pointing: 3843, antenna_name: 1,
                          local_sky_dir_label: 2)
Coordinates:
  * antenna_name         (antenna_name) <U3 12B 'GBT'
  * local_sky_dir_label  (local_sky_dir_label) <U3 24B 'az' 'alt'
  * time_pointing        (time_pointing) float64 31kB 4.575e+09 ... 4.575e+09
Data variables:
    POINTING_BEAM        (time_pointing, antenna_name, local_sky_dir_label) float64 61kB dask.array<chunksize=(3843, 1, 2), meta=np.ndarray>
Attributes:
    type:     pointing